## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [ ]:
from nltk import download

download('movie_reviews')

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [ ]:
from nltk.corpus import movie_reviews

# Total reviews
print (len(movie_reviews.fileids())) # Output: 2000
 
# Review categories
print (movie_reviews.categories()) # Output: [u'neg', u'pos']
 
# Total positive reviews
print (len(movie_reviews.fileids('pos'))) # Output: 1000
 
# Total negative reviews
print (len(movie_reviews.fileids('neg'))) # Output: 1000

In [ ]:
import string
print(string.punctuation)

In [1]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label
import re
regx="[\\n!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~]"

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
#    reviews_pos.append(review)
    review=re.sub(regx, " ",  review)
    review=re.sub("'", "",  review)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    review=re.sub(regx, " ",  review)
    review=re.sub("'", "",  review)
    reviews_neg.append(review)

In [ ]:
reviews_pos[0]


### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [ ]:
!algo run nlp/SentimentAnalysis/1.0.5 -d '{"document": "films adapted from comic books have had plenty of success , whether theyre about superheroes  batman , superman , spawn  , or geared toward kids  casper  or the arthouse crowd  ghost world  , but theres never really been a comic book like from hell before . for starters , it was created by alan moore  and eddie campbell  , who brought the medium to a whole new level in the mid 80s with a 12-part series called the watchmen . to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . the book  or  graphic novel ,  if you will  is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . in other words , dont dismiss this film because of its source . if you can get past the whole comic book thing , you might find another stumbling block in from hells directors , albert and allen hughes . getting the hughes brothers to direct this seems almost as ludicrous as casting carrot top in , well , anything , but riddle me this : who better to direct a film thats set in the ghetto and features really violent street crime than the mad geniuses behind menace ii society ? the ghetto in question is , of course , whitechapel in 1888 londons east end . its a filthy , sooty place where the whores  called  unfortunates   are starting to get a little nervous about this mysterious psychopath who has been carving through their profession with surgical precision . when the first stiff turns up , copper peter godley  robbie coltrane , the world is not enough  calls in inspector frederick abberline  johnny depp , blow  to crack the case . abberline , a widower , has prophetic dreams he unsuccessfully tries to quell with copious amounts of absinthe and opium . upon arriving in whitechapel , he befriends an unfortunate named mary kelly  heather graham , say it isnt so  and proceeds to investigate the horribly gruesome crimes that even the police surgeon cant stomach . i dont think anyone needs to be briefed on jack the ripper , so i wont go into the particulars here , other than to say moore and campbell have a unique and interesting theory about both the identity of the killer and the reasons he chooses to slay . in the comic , they dont bother cloaking the identity of the ripper , but screenwriters terry hayes  vertical limit  and rafael yglesias  les mis ? rables  do a good job of keeping him hidden from viewers until the very end . its funny to watch the locals blindly point the finger of blame at jews and indians because , after all , an englishman could never be capable of committing such ghastly acts . and from hells ending had me whistling the stonecutters song from the simpsons for days   who holds back the electric car/who made steve guttenberg a star ?   . dont worry - itll all make sense when you see it . now onto from hells appearance : its certainly dark and bleak enough , and its surprising to see how much more it looks like a tim burton film than planet of the apes did  at times , it seems like sleepy hollow 2  . the print i saw wasnt completely finished  both color and music had not been finalized , so no comments about marilyn manson  , but cinematographer peter deming  dont say a word  ably captures the dreariness of victorian-era london and helped make the flashy killing scenes remind me of the crazy flashbacks in twin peaks , even though the violence in the film pales in comparison to that in the black-and-white comic . oscar winner martin childs  shakespeare in love  production design turns the original prague surroundings into one creepy place . even the acting in from hell is solid , with the dreamy depp turning in a typically strong performance and deftly handling a british accent . ians holm  joe goulds secret  and richardson  102 dalmatians  log in great supporting roles , but the big surprise here is graham . i cringed the first time she opened her mouth , imagining her attempt at an irish accent , but it actually wasnt half bad . the film , however , is all good . 2 : 00 - r for strong violence/gore , sexuality , language and drug content "}' --timeout 300 

In [ ]:
score_review(reviews_pos[0])

In [2]:
import numpy as np
def score_review(review):
    # TBD: call the service and return 'pos' or 'neg'
    
    yadda =  '{"document": "' + review + '"}'
    yadda = "'" + yadda + "'"
    yadda = 'nlp/SentimentAnalysis/1.0.5 -d '  + yadda +  ' --timeout 300'
    blah = !algo run $yadda
    
    blah=float(blah[1].split(":")[2].split("}")[0])
    
    blah=1/(1+np.exp(-1*blah))
    
    #return float(blah[1].split(":")[2].split("}")[0])

    if blah>0.5:
        blah='pos'
    else:
        blah='neg'
    
    return blah

### Score each review

Now, we can use the function you defined to score each of the reviews.

#### *Note on Testing*

While most of the services listed have free tiers they may be limited to a few thousand requests per week or month, depending on the service. On some platforms you may be billed after reaching that limit. For this reason it is recommended to first test on a smaller set of the reviews, `subset_pos` and `subset_neg`. Once you're happy with your code swap those subsets for the full review sets `reviews_pos` and `reviews_neg`.

In [3]:
# Create 2 smaller subsets for testing
subset_pos = reviews_pos[:10]
subset_neg = reviews_neg[:10]

results_pos = []
# When comfortable with results switch `subset_pos` to reviews_post`
for review in subset_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
# When comfortable with results switch `subset_neg` to reviews_neg`
for review in subset_neg:
    result = score_review(review)
    results_neg.append(result)

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [4]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 60.0% correct
Negative reviews: 40.0% correct
Overall accuracy: 50.0% correct
